Face Detection with Deep Learning

Locating a face in a photograph : finding the coordinate of the face in the image.
Localization - demarcating the extent of the face, often via a bounding box around the face.

2. Using Multi-Task Cascaded Convolutional Neural Network (MTCNN)

In [ ]:
# extract and plot each detected face in a photograph
from matplotlib import pyplot
from matplotlib.patches import Rectangle
from matplotlib.patches import Circle
from mtcnn.mtcnn import MTCNN

In [ ]:

# draw each face separately
def draw_faces(filename, result_list):
	# load the image
	data = pyplot.imread(filename)
	# plot each face as a subplot
	for i in range(len(result_list)):
		# get coordinates
		x1, y1, width, height = result_list[i]['box']
		x2, y2 = x1 + width, y1 + height
		# define subplot
		pyplot.subplot(1, len(result_list), i+1)
		pyplot.axis('off')
		# plot face
		pyplot.imshow(data[y1:y2, x1:x2])
	# show the plot
	pyplot.show()

In [ ]:
def detect_face_using_mtcnn(img_name):
	pixels = cv2.imread(img_name)
	detector = MTCNN()
	faces = detector.detect_faces(pixels)
	print("Number of faces: ",len(faces))
	# draw_faces(filename, faces)


In [ ]:

import cv2
import os
import uuid
import time


In [ ]:

capture = cv2.VideoCapture(0)
img_num=1
while img_num<=10:
	init = time.time()
	print('Collecting image {}'.format(img_num))
	captured,frame =capture.read()
	img_name = os.path.join('./data/images_mtcnn/',f'{str(img_num)}.jpg')
	if not captured:
		print("Frame Not Captured")
	else:
		cv2.imwrite(img_name, frame)
		img_num+=1
		
		time.sleep(0.5)
	
		detect_face_using_mtcnn(img_name)
	
	if cv2.waitKey(1) & 0xFF==ord('q'): break
	fps = 1/(time.time() - init)
	print('Fps: {}'.format(fps))

capture.release()
# close the window 
cv2.destroyAllWindows()

In the above MTCNN technique,
- Detection of inverted faces and tilted faces (A few tilted images could be omitted)
- The Images are written - storage issue
- The detection time needs to be optimized

2.1 Using 'asyncio' python library to fasten the code using asynchronous programming :

In [ ]:
# extract and plot each detected face in a photograph

from matplotlib import pyplot
from matplotlib.patches import Rectangle
from matplotlib.patches import Circle
from mtcnn.mtcnn import MTCNN
from zmq import NULL


In [ ]:

# draw each face separately
def draw_faces(filename, result_list):
	# load the image
	data = pyplot.imread(filename)
	# plot each face as a subplot
	for i in range(len(result_list)):
		# get coordinates
		x1, y1, width, height = result_list[i]['box']
		x2, y2 = x1 + width, y1 + height
		# define subplot
		pyplot.subplot(1, len(result_list), i+1)
		pyplot.axis('off')
		# plot face
		pyplot.imshow(data[y1:y2, x1:x2])
	# show the plot
	pyplot.show()
	


In [ ]:
def detect_face_using_mtcnn(filename):
	#---------------------Check For Exception Handling 
	# load image from file
	pixels = pyplot.imread(filename)
	# create the detector, using default weights
	detector = MTCNN()
	# detect faces in the image
	faces = detector.detect_faces(pixels)
	# display faces on the original image
	print("Number of faces: ",len(faces))
	# draw_faces(filename, faces)


In [ ]:

import cv2
import os
import uuid
import time
import asyncio



In [ ]:


async def captureVideo():
	global img_num,img_name,frame
	task = asyncio.create_task(write())
	print('Collecting image {}'.format(img_num))
	captured,frame =capture.read()
	img_name = os.path.join('./data/images_mtcnn/',f'{str(img_num)}.jpg')
	if not captured:
		print("Frame Not Captured")
	else:
		img_num+=1
		# cv2.imshow('Frame: '+str(img_num), frame)
		await asyncio.sleep(0.5)


In [ ]:

async def write():
	cv2.imwrite(img_name, frame)



In [ ]:
	
capture = cv2.VideoCapture(0)
img_num=1
img_name = ""
frame = NULL
while img_num<=10:
	init = time.time()
	try: 
		loop = asyncio.get_running_loop()
	except RuntimeError:
		loop=None
	if loop and loop.is_running():
		print("----------------------------------------------------------------")
		await captureVideo()
	else:
		asyncio.run(captureVideo())
	# keep the window open until we press a key
	detect_face_using_mtcnn(img_name)
	
	if cv2.waitKey(1) & 0xFF==ord('q'): break
	fps = 1/(time.time() - init)
	print('Fps: {}'.format(fps))

capture.release()
# close the window 
cv2.destroyAllWindows()


2.2 Using Multi-threading :

In [ ]:
from threading import Thread
import cv2, time
from matplotlib import pyplot
from matplotlib.patches import Rectangle
from matplotlib.patches import Circle
from mtcnn.mtcnn import MTCNN
import sys


In [ ]:

class VideoStreamWidget(object):
    def __init__(self, src=0):
        self.capture = cv2.VideoCapture(src)
        # Start the thread to read frames from the video stream
        self.thread = Thread(target=self.update, args=())
        self.thread.daemon = True
        self.thread.start()

    def update(self):
        # Read the next frame from the stream in a different thread
        while True:
            if self.capture.isOpened():
                (self.status, self.frame) = self.capture.read()
                time.sleep(.5)
            else:
                print("No Cam Found!")
                

                

    def write_frame(self,img_name):
        # Display frames in main program
        cv2.imwrite(img_name, self.frame)
        key = cv2.waitKey(1)
        if key == ord('q'):
            self.capture.release()
            cv2.destroyAllWindows()
            exit(1)
    # draw each face separately
    def draw_faces(self,filename, result_list):
        # load the image
        data = pyplot.imread(filename)
        # plot each face as a subplot
        for i in range(len(result_list)):
            # get coordinates
            x1, y1, width, height = result_list[i]['box']
            x2, y2 = x1 + width, y1 + height
            # define subplot
            pyplot.subplot(1, len(result_list), i+1)
            pyplot.axis('off')
            # plot face
            pyplot.imshow(data[y1:y2, x1:x2])
        # show the plot
        pyplot.show()

    def detect_face_using_mtcnn(self,filename):
        print(f"Detecting from : {img_name}")
        #---------------------Check For Exception Handling 
        # load image from file
        pixels = pyplot.imread(filename)
        # create the detector, using default weights
        detector = MTCNN()
        # detect faces in the image
        faces = detector.detect_faces(pixels)
        # display faces on the original image
        num_faces = len(faces)
        print("Number of faces: ",num_faces)
        # draw_faces(filename, faces)
        return faces


In [ ]:

img_num=1
if __name__ == '__main__':
    video_stream_widget = VideoStreamWidget()
    while True:
        try:    
            img_name = os.path.join('./data/images_mtcnn_multithreading/',f'{str(img_num)}.jpg')
            video_stream_widget.write_frame(img_name)
            faces = video_stream_widget.detect_face_using_mtcnn(img_name)
            if len(faces)>1 :
                video_stream_widget.draw_faces(img_name,faces)
            if len(faces)<1 :
                print("No Face Detected")
            img_num+=1
        except AttributeError:
            pass

2.2.1 Addressing the related storage issue using frame processing

In [ ]:
from threading import Thread
import cv2, time
from matplotlib import pyplot
from matplotlib.patches import Rectangle
from matplotlib.patches import Circle
from mtcnn.mtcnn import MTCNN
import sys


In [ ]:

class VideoStreamWidget(object):
    def __init__(self, src=0):
        self.capture = cv2.VideoCapture(src)
        # Start the thread to read frames from the video stream
        self.thread = Thread(target=self.update, args=())
        # self.thread.daemon = True
        self.thread.start()

    def update(self):
        # Read the next frame from the stream in a different thread
        while True:
            if self.capture.isOpened():
                (self.status, self.frame) = self.capture.read()
                self.frame  = cv2.cvtColor(self.frame,cv2.COLOR_BGR2RGB)
                time.sleep(.5)
            else:
                print("No Cam Found!")
                

                

    def write_frame(self,img_name):
        # Display frames in main program
        cv2.imwrite(img_name, self.frame)
        key = cv2.waitKey(1)
        if key == ord('q'):
            self.capture.release()
            cv2.destroyAllWindows()
            exit(1)
    # draw each face separately
    def draw_faces(self,result_list):
        # plot each face as a subplot
        for i in range(len(result_list)):
            # get coordinates
            x1, y1, width, height = result_list[i]['box']
            x2, y2 = x1 + width, y1 + height
            # define subplot
            pyplot.subplot(1, len(result_list), i+1)
            pyplot.axis('off')
            # plot face
            pyplot.imshow(self.frame[y1:y2, x1:x2])
        # show the plot
        pyplot.show()

    def detect_face_using_mtcnn(self):
        print(f"Detecting from Frame {img_num}:")
        #---------------------Check For Exception Handling 
        detector = MTCNN()
        # detect faces in the image
        faces = detector.detect_faces(self.frame)
        # display faces on the original image
        num_faces = len(faces)
        print("Number of faces: ",num_faces)
        # draw_faces(filename, faces)
        return faces


In [ ]:

img_num=1
if __name__ == '__main__':
    video_stream_widget = VideoStreamWidget()
    while True:
        try:    
            # img_name = os.path.join('./data/images_mtcnn_multithreading/',f'{str(img_num)}.jpg')
            # video_stream_widget.write_frame(img_name)
            faces = video_stream_widget.detect_face_using_mtcnn()
            if len(faces)>1 :
                video_stream_widget.draw_faces(faces)
            if len(faces)<1 :
                print("No Face Detected")
            img_num+=1
        except AttributeError:
            pass

The above storage issue minimizal attempt using frame processing led to less efficiency of the detection algorithm 

2.2.2 Addressing the related storage issue using python's 'tempfile' module
